In [1]:
dataset = qml.data.load('qchem', molname="H2", basis=ST)[0]
H, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)

NameError: name 'qml' is not defined

In [7]:
from pennylane import qchem
symbols, coordinates = qchem.read_structure("CH3F.xyz")

In [8]:
H, qubits = qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits: {:}".format(qubits))
print("Qubit Hamiltonian")
print(H)

Number of qubits: 26
Qubit Hamiltonian
  (-82.7701479791045) [I0]
+ (0.7377222655392843) [Z23]
+ (0.7377222655392849) [Z22]
+ (0.7377222916674246) [Z20]
+ (0.7377222916674251) [Z21]
+ (0.7605841028951641) [Z24]
+ (0.7605841028951641) [Z25]
+ (1.373825628673427) [Z19]
+ (1.3738256286734274) [Z18]
+ (1.3832751171789412) [Z6]
+ (1.3832751171789412) [Z7]
+ (1.433448592804239) [Z9]
+ (1.4334485928042395) [Z8]
+ (1.433448676104116) [Z10]
+ (1.433448676104116) [Z11]
+ (1.7850471188743213) [Z17]
+ (1.785047118874322) [Z16]
+ (1.7850472221850342) [Z15]
+ (1.7850472221850344) [Z14]
+ (1.843273989213221) [Z12]
+ (1.8432739892132215) [Z13]
+ (2.5221790666439663) [Z4]
+ (2.5221790666439663) [Z5]
+ (7.03578010639543) [Z3]
+ (7.035780106395432) [Z2]
+ (16.360957896240606) [Z0]
+ (16.360957896240606) [Z1]
+ (-0.007451431248814472) [Y22 Y24]
+ (-0.007451431248814472) [X22 X24]
+ (-0.006652662396565584) [Y5 Y7]
+ (-0.006652662396565584) [X5 X7]
+ (-0.0037796933376183675) [Y7 Y9]
+ (-0.003779693337618367

In [4]:
import pennylane as qml
from pennylane import qchem

# Hartree-Fock state
hf = qml.qchem.hf_state(electrons=2,orbitals=4)

In [5]:
from pennylane import numpy as np

# atomic symbols defining the molecule
symbols = ['H','H']

# list to store energies
energies = []

# set up a loop to change bond length
r_range = np.arange(0.5, 5.0, 0.25)

# keeps track of points in the potential energy surface
pes_point = 0

In [ ]:
for r in r_range:
    # Change only the z coordinate of one atom
    coordinates = np.array([0.0, 0.0, 0.0, 0.0, 0.0, r])

    # Obtain the qubit Hamiltonian
    H, qubits = qchem.molecular_hamiltonian(symbols, coordinates, method='pyscf')

    # define the device, optimizer and circuit
    dev = qml.device("default.qubit", wires=qubits)
    opt = qml.GradientDescentOptimizer(stepsize=0.4)

    @qml.qnode(dev, interface='autograd')
    def circuit(parameters):
        # Prepare the HF state: |1100>
        qml.BasisState(hf, wires=range(qubits))
        qml.DoubleExcitation(parameters[0], wires=[0, 1, 2, 3])
        qml.SingleExcitation(parameters[1], wires=[0, 2])
        qml.SingleExcitation(parameters[2], wires=[1, 3])

        return qml.expval(H)  # we are interested in minimizing this expectation value

    # initialize the gate parameters
    params = np.zeros(3, requires_grad=True)

    # initialize with converged parameters from previous point
    if pes_point > 0:
        params = params_old

    prev_energy = 0.0
    for n in range(50):
        # perform optimization step
        params, energy = opt.step_and_cost(circuit, params)

        if np.abs(energy - prev_energy) < 1e-6:
            break
        prev_energy = energy

    # store the converged parameters
    params_old = params
    pes_point = pes_point + 1

    energies.append(energy)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(r_range, energies)

ax.set(
    xlabel="Bond length (Bohr)",
    ylabel="Total energy (Hartree)",
    title="Potential energy surface for H$_2$ dissociation",
)
ax.grid()
plt.show()